In [ ]:
import numpy as np
import sys
from matplotlib import pyplot as plt
import scipy.stats
from scipy import io
import seaborn
from pyqtgraph.Qt import QtGui, QtCore
#from PySide import QtGui, QtCore
import pyqtgraph as pg
%matplotlib inline

# HELPER CODE AND NOTES

def create_map():
    # Each row represents a line from (first entry, second entry) to
    # (third entry, fourth entry)
    room_map = np.asarray([

        # outer walls
        [0, 0, 0, 1], 
        [0, 1, 1, 1],  
        [1, 1, 1, 0],  
        [1, 0, 0, 0],  

        # upper rooms
        [0,    0.6, 0.15, 0.6],  
        [0.25, 0.6, 0.6,  0.6],  
        [0.7,  0.6, 1.0,  0.6],
        [0.4,  1.0, 0.4,  0.6],

        # lower rooms
        [0,    0.4, 0.15, 0.4],  
        [0.25, 0.4, 0.8,  0.4],  
        [0.9,  0.4, 1.0,  0.4],
        [0.4,  0.0, 0.4,  0.4],

        # "table"
        [0.1, 0.1, 0.1, 0.15],  
        [0.1, 0.15, 0.15, 0.15],  
        [0.15, 0.15, 0.15, 0.1]
    ])

    return room_map

# adapted from the original matlab code at
# http://www.mathworks.com/matlabcentral/fileexchange/
# 27205-fast-line-segment-intersection
def line_intersect( X1,Y1,X2,Y2, X3,Y3,X4,Y4 ):
    # Check to see if the lines from (x1,y1) to (x2,y2) and
    # from (x3,y3) to (x4,y4) intersect
    X4_X3 = X4.T - X3.T
    Y1_Y3 = Y1   - Y3.T
    Y4_Y3 = Y4.T - Y3.T
    X1_X3 = X1   - X3.T
    X2_X1 = X2   - X1
    Y2_Y1 = Y2   - Y1

    numerator_a = X4_X3 * Y1_Y3 - Y4_Y3 * X1_X3
    numerator_b = X2_X1 * Y1_Y3 - Y2_Y1 * X1_X3
    denominator = Y4_Y3 * X2_X1 - X4_X3 * Y2_Y1

    u_a = numerator_a / denominator
    u_b = numerator_b / denominator 

    INT_X = X1 + X2_X1 * u_a
    INT_Y = Y1 + Y2_Y1 * u_a
    INT_B = (u_a >= 0) & (u_a <= 1) & (u_b >= 0) & (u_b <= 1)

    return INT_X, INT_Y, INT_B

def show_map( room_map ):
    for i in range(0,room_map.shape[0]):
        plt.plot([room_map[i,0], room_map[i,2]], [room_map[i,1], room_map[i,3]])

    plt.axis('equal')
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])    

def vec_linspace( a, b, num=10 ):
    return np.asmatrix( [
        np.linspace( a[0,0], b[0,0], num=num ),
        np.linspace( a[0,1], b[0,1], num=num ),
        np.linspace( a[0,2], b[0,2], num=num )        
    ] )

def cast_rays( parts, room_map, verbose=False ):

    sensor_thetas = 0.1 * np.asarray( range(-5,6) )

    px = parts[0:1,:].T
    py = parts[1:2,:].T
    pt = parts[2:3,:].T

    rx1 = room_map[:,0:1]
    ry1 = room_map[:,1:2]
    rx2 = room_map[:,2:3]
    ry2 = room_map[:,3:4]

    dl = []
    for theta in sensor_thetas:
        # calculate intersections between all rays and all map lines
        int_x, int_y, int_b = line_intersect( px, py,
                                              px + np.sin(theta+pt),
                                              py + np.cos(theta+pt),
                                              rx1, ry1, rx2, ry2 )

        inds = int_b.ravel()

        # calculate nearest intersections
        # simplify code by setting non-intersections to be far away
        n_int_b = np.logical_not( int_b )
        int_x[ n_int_b ] = 1000 
        int_y[ n_int_b ] = 1000

        # calculate distances
        dx = px-int_x
        dy = py-int_y
        dists = dx*dx + dy*dy
        min_dists = np.min( dists, axis=1, keepdims=True )
        dl.append( min_dists )

        # show rays from a point, and their intersections
        # only needed for visualization
        if verbose:
            min_pts = np.argmin( dists, axis=1 )
            for i in range( 0, px.shape[0] ):
                plt.plot( [ px[i,0], int_x[i,min_pts[i]] ],
                          [ py[i,0], int_y[i,min_pts[i]] ] )

    data = np.hstack( dl )

    return data


# MY CODE

# INITIALIZATION

sensor_data = io.loadmat('sensors.mat')
data = sensor_data['sonars']
true_states = sensor_data['true_states']
room_map = create_map()

# Show map
app = QtGui.QApplication( [] )
win = pg.GraphicsWindow( title="Particle filter" )
win.resize( 600, 600 )
win.setWindowTitle( 'Particle filter' )
pg.setConfigOptions( antialias=True )
p3 = win.addPlot( title="Room map" )

for i in range( 0, room_map.shape[0] ):
    p3.plot( [room_map[i,0], room_map[i,2]], [room_map[i,1], room_map[i,3]] )
p3.setXRange( -0.1, 1.1 )
p3.setYRange( -0.1, 1.1 ) 

# Create an initial set of particles- blobs in each room + hallway
S = 2500
# PARAMETERS TO TUNE
sigma = .02
theta = .4
Sigma = 25
parts = np.ones((S,3))
"""
centers = np.array([[0.2,0.7,0.2,0.7],[0.2,0.2,0.8,0.8]])
for i in xrange(4):
    parts[i*S/5:(i+1)*S/5,0] = np.random.normal(centers[0,i],0.05,S/5)
    parts[i*S/5:(i+1)*S/5,1] = np.random.normal(centers[1,i],0.05,S/5)
    parts[i*S/5:(i+1)*S/5,2] = np.random.uniform(0,2*np.pi,S/5)
parts[4*S/5:,0] = np.random.uniform(0.01,0.99,S/5)
parts[4*S/5:,1] = np.random.normal(0.5,0.025,S/5)
parts[4*S/5:,2] = np.random.uniform(0,2*np.pi,S/5)
"""
# Blobs in each room did not work
parts[:,0] = np.random.uniform(0,1,S)
parts[:,1] = np.random.uniform(0,1,S)
parts[:,2] = np.random.uniform(0,2*np.pi,S)
#parts[:,0] = np.random.normal(0.2,0.05,S)
#parts[:,1] = np.random.normal(0.8,0.05,S)
#parts[:,2] - np.random.uniform(0,2*np.pi,S)

pts = p3.scatterPlot(parts[:,0],parts[:,1],symbol='o',size=1,pen=(255,100,100))

N = 225 # number of time values t
ts_plot = None
ex_plot = None
exs = np.zeros((3,N))
exs[:,0] = np.sum(parts,axis=0)

def ray_lik(data, part_sensor_data):
    """
    Vectorized likelihood function p(yt|zt).
    
    Returns an Nx1 matrix with the likelihood of each particle
    
    Try a Gaussian first, then replace it with Laplacian since Gaussian doesn't
    work very well
    """
    return np.exp(-1*Sigma*np.sum(np.abs(data - part_sensor_data),axis=0))

# For all of the robot locations
for t in xrange(N):
    """
    Notes:
    For computing the proposal distribution, you literally just do something like
    x += np.random.normal(0,sigma,S)
    
    What to do with angles? ohhhhhh it's the third column, got it
    """
    
    # Add some randomness to let the particles follow
    parts[:,0] += np.random.normal(0,sigma,S)
    parts[:,1] += np.random.normal(0,sigma,S)
    parts[:,2] += np.random.normal(0,theta,S)
    
    # See how close each point is to what the robot sees
    part_sensor_data = cast_rays(parts.T, room_map)
    liks = ray_lik(data[:,t:t+1], part_sensor_data.T).T
        
    # Normalize the weights
    liks = liks/np.sum(liks)
    
    # Check if we need to resample
    S_eff = 1.0/np.sum(liks**2)
    if S_eff < S:
        vals = np.random.uniform(0,1,S)
        cum_liks = np.cumsum(liks)
        inds = np.array([np.argmin(cum_liks < vals[i]) for i in xrange(S)])
        #inds = np.random.choice( range(S), p=liks )
        #print inds.shape
        parts = parts[inds,:]
        exs[:,t] = np.sum(parts,axis=0)/S
        
    # Plot the new particles
    # Plot the robot's path up to time t
    if ts_plot == None: 
        ts_plot = p3.plot( true_states[0,0:t+1], true_states[1,0:t+1], pen=(0,0,255) )
        ex_plot = p3.plot( exs[0,0:t+1], exs[1,0:t+1], pen=(0,255,0) )
    
    else:
        ts_plot.setData( true_states[0,0:t+1], true_states[1,0:t+1] )
        ex_plot.setData( exs[0,0:t+1], exs[1,0:t+1] )
        pts.setData(parts[:,0],parts[:,1])
        
    pg.QtGui.QApplication.processEvents()
    
    
# Make it stop when you quit
sys.exit(app.exec_())

C:\Anaconda\lib\site-packages\pyqtgraph\graphicsItems\ScatterPlotItem.py:186: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.atlasData[x:x+w, y:y+h] = rendered[key]
C:\Anaconda\lib\site-packages\pyqtgraph\graphicsItems\ScatterPlotItem.py:148: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  arr = self.atlasData[x:x+w, y:y+w]


In [ ]:
from PIL import Image
Image.open("room_map.png").show()